# Agente de IA para Suprimentos: Navegando Relações com Grafos

Este notebook demonstra a construção de um agente de inteligência artificial capaz de responder perguntas sobre contratos de fornecimento com base em dados do Banco Mundial. Utilizando técnicas de busca em grafos, o agente é projetado para explorar e interpretar relações entre entidades como fornecedores, contratos, países e categorias de compras, oferecendo uma interface inteligente para análise e tomada de decisão na área de suprimentos.

A solução combina o modelo de linguagem da OpenAI para interpretação de perguntas e geração de respostas em linguagem natural, com o banco de dados em grafos Neo4j Aura para armazenamento e consulta das relações complexas entre os dados. Essa integração permite criar um agente capaz de compreender o contexto, navegar estruturas de dados conectadas e fornecer respostas contextualizadas e explicáveis.

## 0. Preparação do notebook

### 0.1 Download das bibliotecas

In [ ]:
%pip install requests neo4j tqdm python-dotenv openai xmltodict tiktoken

### 0.2 Definição de funções utilitárias

Função para calular a quantidade de tokens de um texto

In [2]:
import tiktoken

def calculate_token_count(context):
    encoding = tiktoken.encoding_for_model('gpt-4o')
    token_count = len(encoding.encode(context))
    return token_count

Função para calcular o custo de uma chamada de LLM com base nos tokens consumidos

In [3]:
def calculate_request_cost(model, input_tokens=0, output_tokens=0):
    mapper = {
        'gpt-4o': {'input': 2.5/1000000, 'output': 10/1000000},
        'gpt-4.1-nano': {'input': 0.1/1000000, 'output': 0.4/1000000}
    }
    return (mapper[model]['input'] * input_tokens + mapper[model]['output'] * output_tokens) * 5.5

Função para calcular o custo potencial mensal de uma empresa para uma request

In [4]:
def calculate_monthly_potencial_cost(request_cost, n_daily_requests=0, n_employee=0, n_monthly_work_days=22):
    return request_cost * n_daily_requests * n_employee * n_monthly_work_days

## 1. Preparação dos Dados

Nesta etapa, realizamos a obtenção e o pré-processamento dos dados de contratos de fornecimento disponibilizados pelo Banco Mundial. O objetivo é transformar os dados brutos em um formato estruturado e adequado para inserção no banco de dados em grafos.

### 1.1 Extração dos dados

Os dados são obtidos diretamente da fonte oficial via uma requisição HTTP ao arquivo JSON publicado pelo Banco Mundial. Após o download, o arquivo é salvo localmente para uso nas etapas seguintes.

In [5]:
import requests
import json

url = 'https://datacatalogapi.worldbank.org/dexapps/fone/api/apiservice?datasetId=DS00028&resourceId=RS00025&type=json'

response = requests.get(url)

with open(f'data/raw_dataset.json', 'w', encoding='utf-8') as file:
    json.dump(response.json(), file, ensure_ascii=False)

print('Data saved at: data/raw_dataset.json')
print(response.json())

Data saved at: data/raw_dataset.json
{'count': 3784, 'data': [{'award_date': '24-Dec-2024', 'commodity_category': 'CONSULTING', 'contract_award_amount': 480000, 'contract_description': 'Recommendation for Service Plan, Concept Design, & Econ Analysis of PT Corridor in Baghdad', 'fund_source': 'Trust Fund', 'quarter_and_fiscal_year': 'Q2 - FY25', 'selection_number': '2009796', 'supplier': 'CONSULTRANS, S.A.U.', 'supplier_country': 'Spain', 'supplier_country_code': 'ES', 'vpu_description': 'Office of the Regional Vice Pres', 'wbg_organization': 'IBRD'}, {'award_date': '24-Dec-2024', 'commodity_category': 'CONSULTING', 'contract_award_amount': 250000, 'contract_description': 'Design, Plan and Launch a Food System Global Innovation Acceleration Program and its Associated Fund', 'fund_source': 'Trust Fund', 'quarter_and_fiscal_year': 'Q2 - FY25', 'selection_number': '2012029', 'supplier': 'World Food Programme', 'supplier_country': 'Italy', 'supplier_country_code': 'IT', 'vpu_description': 

### 1.2. Transformação dos dados

Após a extração, o arquivo JSON é lido e transformado para garantir que os dados estejam em um formato limpo e consistente. As principais transformações realizadas incluem:

Conversão da data de adjudicação (award_date) para o formato ISO (YYYY-MM-DD)

Tratamento de caracteres especiais no campo de descrição do contrato, garantindo compatibilidade com o formato JSON e posterior ingestão em banco de dados

Abaixo está o código utilizado para realizar essas transformações:

In [6]:
from datetime import datetime
import json

with open('data/raw_dataset.json', encoding='utf-8') as file:
    data = json.load(file)['data']

for item in data:
    parsed_date = datetime.strptime(item["award_date"], "%d-%b-%Y")
    item["award_date"] = parsed_date.strftime("%Y-%m-%d")
    item["contract_description"] = item["contract_description"].replace('\\', '\\\\').replace('"', '\\"').replace("'", "\\'")

with open(f'data/treated_dataset.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, indent=4, ensure_ascii=False)

print('Data saved at: data/treated_dataset.json')
print(data)

Data saved at: data/treated_dataset.json
[{'award_date': '2024-12-24', 'commodity_category': 'CONSULTING', 'contract_award_amount': 480000, 'contract_description': 'Recommendation for Service Plan, Concept Design, & Econ Analysis of PT Corridor in Baghdad', 'fund_source': 'Trust Fund', 'quarter_and_fiscal_year': 'Q2 - FY25', 'selection_number': '2009796', 'supplier': 'CONSULTRANS, S.A.U.', 'supplier_country': 'Spain', 'supplier_country_code': 'ES', 'vpu_description': 'Office of the Regional Vice Pres', 'wbg_organization': 'IBRD'}, {'award_date': '2024-12-24', 'commodity_category': 'CONSULTING', 'contract_award_amount': 250000, 'contract_description': 'Design, Plan and Launch a Food System Global Innovation Acceleration Program and its Associated Fund', 'fund_source': 'Trust Fund', 'quarter_and_fiscal_year': 'Q2 - FY25', 'selection_number': '2012029', 'supplier': 'World Food Programme', 'supplier_country': 'Italy', 'supplier_country_code': 'IT', 'vpu_description': 'Planet Vice Presidenc

### 1.3 Avaliação do contexto

In [8]:
with open(f'data/treated_dataset.json', encoding='utf-8') as file:
    context = file.read()

ref_token_count = calculate_token_count(context)
ref_request_cost = calculate_request_cost('gpt-4o', ref_token_count)
ref_potencial_cost =  calculate_monthly_potencial_cost(ref_request_cost, 15, 120, 22)

print('Contagem de tokens:', ref_token_count)
print('Custo da requisição (R$):', round(ref_request_cost, 2))
print('Custo mensal potencial (R$):', ref_potencial_cost)


Contagem de tokens: 146680
Custo da requisição (R$): 2.02
Custo mensal potencial (R$): 79867.26000000001


## 2. Modelagem e Construção do Grafo

Nesta etapa, os dados tratados são representados em um grafo no Neo4j, permitindo consultas complexas e navegação eficiente entre entidades relacionadas. A modelagem é orientada a representar os principais elementos do processo de contratação e suas conexões.

### 2.1 Estrutura do Grafo

A modelagem foi construída com os seguintes nós (entidades):
* :`Supplier` – Fornecedor do contrato
* :`Country` – País onde o fornecedor está localizado
* :`CommodityCategory` – Categoria do bem ou serviço contratado
* :`FundSource` – Fonte de financiamento do contrato
* :`Organization` – Organização do Grupo Banco Mundial (WBG) responsável
* :`VPU` – Unidade da Vice-Presidência que gerencia o contrato
* :`Contract` – Objeto do contrato propriamente dito

E os relacionamentos entre essas entidades:
* `(Supplier)-[:LOCATED_IN]->(Country)`
* `(Supplier)-[:PROVIDED]->(Contract)`
* `(Contract)-[:CATEGORIZED_AS]->(CommodityCategory)`
* `(Contract)-[:FUNDED_BY]->(FundSource)`
* `(Contract)-[:MANAGED_BY]->(VPU)`
* `(VPU)-[:PART_OF]->(Organization)`

### 2.2 Conexão com o banco de dados Neo4j

In [9]:
import os
from neo4j import GraphDatabase
from dotenv import load_dotenv; load_dotenv()

uri = os.getenv('NEO4J_URI')
user = os.getenv('NEO4J_USERNAME')
password = os.getenv('NEO4J_PASSWORD')

driver = GraphDatabase.driver(uri, auth=(user, password))

driver.verify_connectivity()

### 2.3 Leitura dos dados tratados

In [10]:
import json

with open('data/treated_dataset.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

### 2.4 Ingestão dos dados no Grafo

Para cada item do dataset, os dados são convertidos em comandos Cypher com base em um template. As inserções são feitas utilizando a operação MERGE, que evita duplicação de nós e relacionamentos:

In [11]:
from tqdm import tqdm

cypher_template = (
    'MERGE (s:Supplier {{name: "{supplier}"}}) '
    'MERGE (c:Country {{name: "{supplier_country}", code: "{supplier_country_code}"}}) '
    'MERGE (cat:CommodityCategory {{name: "{commodity_category}"}}) '
    'MERGE (fs:FundSource {{name: "{fund_source}"}}) '
    'MERGE (org:Organization {{name: "{wbg_organization}"}}) '
    'MERGE (vpu:VPU {{name: "{vpu_description}"}}) '
    'MERGE (s)-[:LOCATED_IN]->(c) '
    'MERGE (vpu)-[:PART_OF]->(org) '
    'MERGE (con:Contract {{selection_number: "{selection_number}"}}) '
    'ON CREATE SET '
    '  con.award_date = date("{award_date}"), '
    '  con.amount = toFloat("{contract_award_amount}"), '
    '  con.description = "{contract_description}", '
    '  con.fiscal_quarter = "{quarter_and_fiscal_year}" '
    'MERGE (s)-[:PROVIDED]->(con) '
    'MERGE (con)-[:CATEGORIZED_AS]->(cat) '
    'MERGE (con)-[:FUNDED_BY]->(fs) '
    'MERGE (con)-[:MANAGED_BY]->(vpu) '
)

failed = []
with driver.session() as session:
    for item in tqdm(data, ncols=100, desc='Uploading data to neo4j'):
        try:
            cleaned_item = {k: (v if v is not None else "") for k, v in item.items()}
            cypher_query = cypher_template.format(**cleaned_item)
            session.execute_write(lambda tx: tx.run(cypher_query))
        except Exception as e:
            print(e)
            failed.append(item)

Uploading data to neo4j: 100%|██████████████████████████████████| 1000/1000 [08:27<00:00,  1.97it/s]


### 2.5 Consulta de verificação

Após a carga, uma consulta simples pode ser executada para validar o sucesso da ingestão. Por exemplo, listar contratos fornecidos por empresas localizadas na Espanha:

In [12]:
cypher = """
MATCH (s:Supplier)-[LOCATED_IN]->(c:Country {name: 'Spain'})
MATCH (s)-[PROVIDED]->(contract:Contract)
RETURN contract, s, c
"""

with driver.session() as session:
    result = session.run(cypher)
    result = [record.data() for record in result]

for item in result:
    print(item)

{'contract': {'description': 'IFC LAC Assessments', 'award_date': neo4j.time.Date(2024, 10, 18), 'fiscal_quarter': 'Q2 - FY25', 'selection_number': '2006830'}, 's': {'name': 'Arup Latin America S.A'}, 'c': {'code': 'ES', 'name': 'Spain'}}
{'contract': {'amount': 304128.0, 'description': 'Consultancy for the Improvement of Five Water Treatment Plants in Guatemala City', 'award_date': neo4j.time.Date(2024, 8, 28), 'fiscal_quarter': 'Q1 - FY25', 'selection_number': '2008510'}, 's': {'name': 'MERCADOS ARIES INTERNATIONAL S.A.'}, 'c': {'code': 'ES', 'name': 'Spain'}}
{'contract': {'amount': 332785.0, 'description': 'Chad : Mini grid - Technical, Environmental and Social Support', 'award_date': neo4j.time.Date(2024, 6, 28), 'fiscal_quarter': 'Q4 - FY24', 'selection_number': '2008254'}, 's': {'name': 'Trama TecnoAmbiental'}, 'c': {'code': 'ES', 'name': 'Spain'}}
{'contract': {'amount': 471865.0, 'description': 'Ecuador(Galapagos) : Harnessing the Potential of the Ocean for Sustainable Develop

## 3. Criação do Agente

Nesta etapa, desenvolvemos um agente que permite ao usuário fazer perguntas em linguagem natural sobre os contratos de fornecimento, e obtém as respostas executando consultas no banco de grafos Neo4j. O agente utiliza o modelo GPT da OpenAI para interpretar a pergunta, gerar a consulta Cypher correspondente, executá-la e retornar os resultados.

### 3.1 Conexão com o banco para consultar ao Grafo

Nesta seção, definimos a função responsável por executar buscas no banco de dados Neo4j. Essa função será usada como ferramenta (tool) pelo agente para consultar o grafo com base em queries Cypher.

Conexão com o banco de dados do Neo4j:

In [22]:
import os
from neo4j import GraphDatabase
from dotenv import load_dotenv; load_dotenv()

uri = os.getenv('NEO4J_URI')
user = os.getenv('NEO4J_USERNAME')
password = os.getenv('NEO4J_PASSWORD')

driver = GraphDatabase.driver(uri, auth=(user, password))

driver.verify_connectivity()

Criação da função que consulta o banco:

In [24]:
def execute_query(cypher: str) -> list:
    with driver.session() as session:
        result = session.run(cypher)
        return [record.data() for record in result]

### 3.2 Definição do contexto utilizado pelo Agente

Nesta seção, apresentamos o contexto operacional do agente de IA, detalhando o system prompt que define seu comportamento

In [29]:
INSTRUCTIONS = """\
<goal>
Criar uma query Cypher para obter dados de um grafo do Neo4j
</goal>

<db-context>
Nós (labels):
  - :Supplier - Fornecedor do contrato - [name]
  - :Country - País onde o fornecedor está localizado - [name, code]
  - :CommodityCategory - Categoria do bem ou serviço contratado - [name]
  - :FundSource - Fonte de financiamento do contrato - [name]
  - :Organization - Organização do Grupo Banco Mundial (WBG) responsável - [name]
  - :VPU - Vice-Presidência que gerencia o contrato - [name]
  - :Contract - Contrato propriamente dito - [selection_number, award_date, amount, description, fiscal_quarter]
</db-context>

<relationship-catalog>
(Supplier)-[:LOCATED_IN]->(Country)
(Supplier)-[:PROVIDED]->(Contract)
(Contract)-[:CATEGORIZED_AS]->(CommodityCategory)
(Contract)-[:FUNDED_BY]->(FundSource)
(Contract)-[:MANAGED_BY]->(VPU)
(VPU)-[:PART_OF]->(Organization)
</relationship-catalog>

<output-format>
<root>
<cot>
CHAIN OF THOUGHT USADA PARA MONTAR A QUERY
</cot>
<cypher>
COMANDO A SER EXECUTADO NO CYPHER
</cypher>
</root>
</output-format>

<remember>
Os dados no grafo estão todos em inglês
</remember>\
"""


### 3.3 Conexão com a Responses API da OpenAI

Nesta seção, implementamos a função responsável por enviar a pergunta do usuário à API da OpenAI, utilizando o system prompt e o esquema de tools definidos anteriormente. O modelo é configurado para interpretar a pergunta e, se necessário, chamar a função executar_query com uma consulta Cypher gerada automaticamente.

In [30]:
from openai import OpenAI
from dotenv import load_dotenv; load_dotenv()

client = OpenAI()

def invoke(input_message:str, instructions:str=None):
    response = client.responses.create(
        model="gpt-4.1-nano",
        input=input_message,
        instructions=instructions
    )
    return response


### 3.4 Execução do Agente

In [31]:
import xmltodict

input_message = 'Quais contratos foram fornecidos por fornecedores espanhóis?'

response = invoke(input_message, INSTRUCTIONS)

parsed_response = xmltodict.parse(response.output_text)['root']
cot = parsed_response['cot']
cypher = parsed_response['cypher']
results = execute_query(cypher)

print('Chain of Thought:', cot, sep='\n', end='\n\n')
print('Comando Cypher:', cypher, sep='\n', end='\n\n')
print('Resultados:')
for it in results:
    print(it)



Chain of Thought:
Para obter os contratos fornecidos por fornecedores espanhóis, preciso inicialmente identificar os fornecedores cujo país é Espanha, usando uma correspondência de país através do relacionamento LOCATED_IN. Depois, relaciono esses fornecedores com seus contratos através do relacionamento PROVIDED. Assim, seleciono apenas os contratos ligados a esses fornecedores espanhóis.

Comando Cypher:
MATCH (supplier:Supplier)-[:LOCATED_IN]->(country:Country {name: 'Spain'})
MATCH (supplier)-[:PROVIDED]->(contract:Contract)
RETURN contract

Resultados:
{'contract': {'description': 'IFC LAC Assessments', 'award_date': neo4j.time.Date(2024, 10, 18), 'fiscal_quarter': 'Q2 - FY25', 'selection_number': '2006830'}}
{'contract': {'amount': 304128.0, 'description': 'Consultancy for the Improvement of Five Water Treatment Plants in Guatemala City', 'award_date': neo4j.time.Date(2024, 8, 28), 'fiscal_quarter': 'Q1 - FY25', 'selection_number': '2008510'}}
{'contract': {'amount': 332785.0, 'd

### 3.5 Avaliação do contexto

In [ ]:
search_input_token_count = calculate_token_count(INSTRUCTIONS)
search_output_token_count = calculate_token_count(response.output_text)
search_request_cost = calculate_request_cost('gpt-4.1-nano', search_input_token_count, search_output_token_count)
search_potential_cost = calculate_monthly_potencial_cost(search_request_cost, 15, 120, 22)

res_input_token_count = calculate_token_count(str(results))
res_request_cost = calculate_request_cost('gpt-4o', res_input_token_count)
res_potential_cost = calculate_monthly_potencial_cost(res_request_cost, 15, 120, 22)

opt_token_count = res_input_token_count + search_input_token_count + search_output_token_count
opt_request_cost = search_request_cost + res_request_cost
opt_potential_cost = search_potential_cost + res_potential_cost

print('Contagem de tokens:', opt_token_count)
print('Custo da requisição (R$):', round(opt_request_cost, 2))
print('Custo mensal potencial (R$):', opt_potential_cost)
print('Redução de custo (%):', round((opt_potential_cost / ref_potencial_cost) - 1 * 100, 2))

Contagem de tokens: 2118
Custo da requisição (R$): 0.02
Custo mensal potencial (R$): 936.3222000000001
Redução de custo (%): -99.99
